In [1]:
import geopandas as gpd

blocks = gpd.read_file('./data/blocks.geojson')
crs = blocks.estimate_utm_crs()
blocks = blocks.to_crs(crs)
blocks

,id,Тип функциональной зоны,isAvailable,geometry
0,PZZ-468,2,True,"POLYGON ((387861.999 6644712.938, 387898.372 6..."
1,PZZ-469,2,True,"POLYGON ((387898.372 6644768.891, 387934.746 6..."
2,PZZ-470,2,True,"POLYGON ((387934.746 6644824.845, 388202.551 6..."
3,PZZ-471,2,True,"POLYGON ((388348.842 6645181.924, 388955.181 6..."
4,PZZ-472,2,True,"POLYGON ((388955.181 6644954.448, 389507.545 6..."
...,...,...,...,...
64,PZZ-532,2,True,"POLYGON ((388395.647 6644500.357, 388242.013 6..."
65,PZZ-533,2,True,"POLYGON ((388650.756 6644532.758, 388496.456 6..."
66,PZZ-534,2,True,"POLYGON ((388242.013 6644608.520, 388088.277 6..."
67,PZZ-535,2,True,"POLYGON ((388496.456 6644626.560, 388336.118 6..."


In [2]:
# from lu_igi.models.land_use import LandUse

# land_use_ids_mapping = {
#     1 : LandUse.RESIDENTIAL,
#     2 : LandUse.RECREATION,
#     3 : LandUse.SPECIAL,
#     4 : LandUse.INDUSTRIAL,
#     5 : LandUse.AGRICULTURE,
#     6 : LandUse.TRANSPORT,
#     7 : LandUse.BUSINESS,
#     10 : LandUse.RESIDENTIAL,
#     11 : LandUse.RESIDENTIAL,
#     12 : LandUse.RESIDENTIAL,
#     13 : LandUse.RESIDENTIAL,
# }

In [3]:
from lu_igi.preprocessing.graph import generate_adjacency_graph, save_graph, load_graph

blocks['land_use'] = None
graph = generate_adjacency_graph(blocks)

2025-02-14 15:36:26.802 | INFO     | lu_igi.preprocessing.graph:generate_adjacency_graph:65 - Validating input
2025-02-14 15:36:26.824 | INFO     | lu_igi.preprocessing.graph:_generate_adjacency_edges:47 - Generating edges
100%|██████████| 484/484 [00:00<00:00, 11403.01it/s]
2025-02-14 15:36:26.906 | SUCCESS  | lu_igi.preprocessing.graph:generate_adjacency_graph:79 - Graph successfully generated


In [4]:
from lu_igi.optimization.optimizer import Optimizer

optimizer = Optimizer(graph)

100%|██████████| 484/484 [00:00<00:00, 91649.80it/s]


In [5]:
from lu_igi.optimization.problem import FitnessType
from lu_igi.models.land_use import LandUse

target_lu_shares = {
    LandUse.RESIDENTIAL : 0.5,
    LandUse.RECREATION : 0.3,
    LandUse.BUSINESS : 0.2
}

blocks_ids = list(blocks.index)
result_df = optimizer.run(blocks_ids, target_lu_shares, n_eval=1000, verbose=False)

In [6]:
optimizer.expand_result_df(result_df)[0]['gdf']

,geometry,assigned_land_use
0,"POLYGON ((387861.999 6644712.938, 387898.372 6...",LandUse.RECREATION
1,"POLYGON ((387898.372 6644768.891, 387934.746 6...",LandUse.RESIDENTIAL
2,"POLYGON ((387934.746 6644824.845, 388202.551 6...",LandUse.RECREATION
3,"POLYGON ((388348.842 6645181.924, 388955.181 6...",LandUse.RECREATION
4,"POLYGON ((388955.181 6644954.448, 389507.545 6...",LandUse.BUSINESS
...,...,...
64,"POLYGON ((388395.647 6644500.357, 388242.013 6...",LandUse.AGRICULTURE
65,"POLYGON ((388650.756 6644532.758, 388496.456 6...",LandUse.TRANSPORT
66,"POLYGON ((388242.013 6644608.520, 388088.277 6...",LandUse.RESIDENTIAL
67,"POLYGON ((388496.456 6644626.560, 388336.118 6...",LandUse.RECREATION
